<a href="https://colab.research.google.com/github/Paridhi-Mittal/SOC_computervision_23b0387/blob/main/WEEK_5_TRANSFER_LEARNING(Image_Detection).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
import tensorflow as tf

In [60]:
from tensorflow import keras
from keras.layers import Dense,Conv2D,MaxPool2D,Flatten
from keras.models import Sequential
from keras.datasets import mnist

In [3]:
(X_train,y_train),(X_test,y_test)=mnist.load_data()


11490434/11490434 [==============================] - 0s 0us/step


In [6]:
X_train.dtype

dtype('uint8')

In [18]:
X_train=X_train.astype('float32')
X_test=X_test.astype('float32')


In [28]:
X_train.shape


(28, 28, 60000)

In [29]:
X_test.shape

(28, 28, 10000)

In [30]:
X_train = X_train.reshape(X_train.shape[2],28,28)
X_test = X_test.reshape(X_test.shape[2],28,28)
import numpy as np


In [32]:
X_train = np.stack((X_train)*3, axis =-1)
X_train.shape


(28, 60000, 28)

In [61]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [62]:
X_train.shape, X_test.shape

((60000, 28, 28), (10000, 28, 28))

In [63]:
X_train = X_train.reshape((X_train.shape[0], 28, 28))
X_test = X_test.reshape((X_test.shape[0], 28, 28))
# change the type to float
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# convert data to 3 channels
X_train = np.stack((X_train,)*3, axis=-1)
X_test = np.stack((X_test,)*3, axis=-1)


In [64]:
y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [65]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

In [66]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [67]:
train_generator = ImageDataGenerator(
    rescale = 1./255,  # normalization of images
    rotation_range = 40, # augmention of images to avoid overfitting
    shear_range = 0.2,
    zoom_range = 0.2,
    fill_mode = 'nearest'
)

val_generator = ImageDataGenerator(rescale = 1./255)

train_iterator = train_generator.flow(X_train, y_train, batch_size=512, shuffle=True)

val_iterator = val_generator.flow(X_test, y_test, batch_size=512, shuffle=False)

In [72]:
from tensorflow.keras.applications.vgg16 import VGG16

In [74]:
conv_base = VGG16(weights='imagenet',
                  include_top=False)


In [75]:
model = Sequential()
model.add(conv_base)
model.add(Flatten())

model.add(Dense(10, activation='softmax'))
conv_base.trainable = False

ValueError: The last dimension of the inputs to a Dense layer should be defined. Found None. Full input shape received: (None, None)

In [76]:
from tensorflow.keras.applications.resnet50 import ResNet50
model = Sequential()
# add the pretrained model
model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))
# add fully connected layer with output
model.add(Dense(512, activation='relu'))
model.add(Dense(10, activation='softmax'))

# set resnet layers not trainable
model.layers[0].trainable=False
model.summary()


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense_14 (Dense)            (None, 512)               1049088   
                                                                 
 dense_15 (Dense)            (None, 10)                5130      
                                                                 
Total params: 24641930 (94.00 MB)
Trainable params: 1054218 (4.02 MB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


In [77]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [78]:
model.fit(train_iterator, epochs=10, validation_data=val_iterator)

Epoch 1/10
118/118 [==============================] - 280s 2s/step - loss: 1.4779 - accuracy: 0.5329 - val_loss: 0.7854 - val_accuracy: 0.7884
Epoch 2/10
118/118 [==============================] - 267s 2s/step - loss: 0.9534 - accuracy: 0.7067 - val_loss: 0.6599 - val_accuracy: 0.7970
Epoch 3/10
118/118 [==============================] - 265s 2s/step - loss: 0.7891 - accuracy: 0.7554 - val_loss: 0.5440 - val_accuracy: 0.8300
Epoch 4/10
118/118 [==============================] - 277s 2s/step - loss: 0.6986 - accuracy: 0.7825 - val_loss: 0.5178 - val_accuracy: 0.8332
Epoch 5/10
118/118 [==============================] - 267s 2s/step - loss: 0.6338 - accuracy: 0.8016 - val_loss: 0.4645 - val_accuracy: 0.8505
Epoch 6/10
118/118 [==============================] - 266s 2s/step - loss: 0.5937 - accuracy: 0.8141 - val_loss: 0.4208 - val_accuracy: 0.8681
Epoch 7/10
118/118 [==============================] - 274s 2s/step - loss: 0.5638 - accuracy: 0.8203 - val_loss: 0.4144 - val_accuracy: 0.8654